# 🧪 Notebook 05A: Feature Engineering A/B Test

**Objective:** Determine if engineered features improve model performance

**Critical Question:** Should we use raw features OR raw + engineered features?

**Strategy:**
- **Set A (Raw):** Original 18 features only
- **Set B (Engineered):** Raw + BMI categories + Risk scores + Key interactions
- **Test Model:** RandomForest with balanced weights (fast baseline)
- **Decision Criteria:** If Set B improves Recall by >2%, use engineered features

**Why This Matters:**
- No point optimizing models on wrong features
- Feature engineering can help OR hurt (overfitting risk)
- Must validate with data, not assumptions

---

## 📦 Step 1: Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    recall_score, precision_score, f1_score, roc_auc_score,
    confusion_matrix, classification_report
)
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Setup complete")

## 📊 Step 2: Load Data & Create Binary Target

In [ ]:
print("=" * 60)
print("LOADING DATA & DEFINING BINARY TARGET")
print("=" * 60)

# Load dataset
df = pd.read_csv('dataset_A_clean.csv')

print(f"\n📊 Dataset loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")

# Create binary target (from Phase 1 decision)
# Class 0: Healthy (Diabetes_012 == 0)
# Class 1: At Risk (Diabetes_012 == 1 OR 2)
y = (df['Diabetes_012'] >= 1).astype(int)

print(f"\n📊 Binary Target Distribution:")
class_counts = y.value_counts().sort_index()
print(f"   Class 0 (Healthy):  {class_counts[0]:7,} ({class_counts[0]/len(y)*100:.2f}%)")
print(f"   Class 1 (At Risk):  {class_counts[1]:7,} ({class_counts[1]/len(y)*100:.2f}%)")
print(f"   Imbalance ratio: {class_counts[0]/class_counts[1]:.1f}:1")

# Store original features (excluding target)
X_raw = df.drop('Diabetes_012', axis=1)
original_features = X_raw.columns.tolist()

print(f"\n📋 Original Features ({len(original_features)}):")
for i, feat in enumerate(original_features, 1):
    print(f"   {i:2d}. {feat}")

## 🔧 Step 3: Create Feature Set A (Raw Features Only)

In [ ]:
print("=" * 60)
print("SET A: RAW FEATURES")
print("=" * 60)

X_set_A = X_raw.copy()

print(f"\n📊 Set A Shape: {X_set_A.shape}")
print(f"   Features: {X_set_A.shape[1]}")
print(f"   Samples: {X_set_A.shape[0]:,}")

print(f"\n✅ Set A ready (baseline raw features)")

## 🛠️ Step 4: Create Feature Set B (Raw + Engineered)

In [ ]:
print("=" * 60)
print("SET B: RAW + ENGINEERED FEATURES")
print("=" * 60)

X_set_B = X_raw.copy()

print(f"\n🔧 Engineering new features...\n")

# ============================================
# 1. BMI Categories (clinical thresholds)
# ============================================
print("1️⃣ BMI Categories:")
X_set_B['BMI_Underweight'] = (X_set_B['BMI'] < 18.5).astype(int)
X_set_B['BMI_Normal'] = ((X_set_B['BMI'] >= 18.5) & (X_set_B['BMI'] < 25)).astype(int)
X_set_B['BMI_Overweight'] = ((X_set_B['BMI'] >= 25) & (X_set_B['BMI'] < 30)).astype(int)
X_set_B['BMI_Obese'] = (X_set_B['BMI'] >= 30).astype(int)
print(f"   ✓ Added 4 BMI category indicators")

# ============================================
# 2. Metabolic Risk Score (key risk factors)
# ============================================
print("\n2️⃣ Metabolic Risk Score:")
X_set_B['Metabolic_Risk_Score'] = (
    X_set_B['HighBP'].astype(int) + 
    X_set_B['HighChol'].astype(int) + 
    (X_set_B['BMI'] >= 30).astype(int)  # Obesity
)
print(f"   ✓ Score = HighBP + HighChol + Obesity (0-3 range)")
print(f"   Distribution:")
print(X_set_B['Metabolic_Risk_Score'].value_counts().sort_index())

# ============================================
# 3. Cardiovascular History (composite)
# ============================================
print("\n3️⃣ Cardiovascular History:")
X_set_B['Cardiovascular_History'] = (
    X_set_B['HeartDiseaseorAttack'].astype(int) + 
    X_set_B['Stroke'].astype(int)
)
print(f"   ✓ Any heart disease or stroke history")

# ============================================
# 4. Age-BMI Interaction (critical for diabetes)
# ============================================
print("\n4️⃣ Age × BMI Interaction:")
X_set_B['Age_BMI_Interaction'] = X_set_B['Age'] * X_set_B['BMI']
print(f"   ✓ Captures compound risk (older + heavier = higher risk)")

# ============================================
# 5. Lifestyle Health Score (protective factors)
# ============================================
print("\n5️⃣ Lifestyle Health Score:")
X_set_B['Lifestyle_Health_Score'] = (
    X_set_B['PhysActivity'].astype(int) + 
    X_set_B['Fruits'].astype(int) + 
    X_set_B['Veggies'].astype(int) + 
    (1 - X_set_B['Smoker'].astype(int))  # Non-smoker is positive
)
print(f"   ✓ Healthy behaviors score (0-4 range)")

# ============================================
# 6. Mental Health Risk Flag
# ============================================
print("\n6️⃣ Mental Health Risk:")
X_set_B['MentHlth_Risk'] = (X_set_B['MentHlth'] >= 15).astype(int)  # >15 days = risk
print(f"   ✓ Flag for poor mental health (≥15 days)")

# ============================================
# 7. Healthcare Access Score
# ============================================
print("\n7️⃣ Healthcare Access:")
X_set_B['Healthcare_Access'] = (
    X_set_B['AnyHealthcare'].astype(int) + 
    X_set_B['CholCheck'].astype(int)
)
print(f"   ✓ Access to care + preventive screening")

print("\n" + "=" * 60)
print(f"📊 Set B Shape: {X_set_B.shape}")
print(f"   Original features: {len(original_features)}")
print(f"   Engineered features: {X_set_B.shape[1] - len(original_features)}")
print(f"   Total features: {X_set_B.shape[1]}")

# List new features
new_features = [col for col in X_set_B.columns if col not in original_features]
print(f"\n📋 Engineered Features ({len(new_features)}):")
for i, feat in enumerate(new_features, 1):
    print(f"   {i:2d}. {feat}")

print(f"\n✅ Set B ready (raw + engineered features)")

## 🔀 Step 5: Train-Test Split (Same for Both Sets)

In [ ]:
print("=" * 60)
print("CREATING TRAIN-TEST SPLITS")
print("=" * 60)

# Split Set A
X_train_A, X_test_A, y_train_A, y_test_A = train_test_split(
    X_set_A, y, test_size=0.2, random_state=42, stratify=y
)

# Split Set B (same random_state = same split)
X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(
    X_set_B, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n📊 Set A:")
print(f"   Train: {X_train_A.shape[0]:,} samples × {X_train_A.shape[1]} features")
print(f"   Test:  {X_test_A.shape[0]:,} samples × {X_test_A.shape[1]} features")

print(f"\n📊 Set B:")
print(f"   Train: {X_train_B.shape[0]:,} samples × {X_train_B.shape[1]} features")
print(f"   Test:  {X_test_B.shape[0]:,} samples × {X_test_B.shape[1]} features")

# Verify same samples
print(f"\n✅ Both sets use SAME train/test split (random_state=42)")
print(f"   This ensures fair comparison")

## 🧪 Step 6: Test Set A (Raw Features)

In [ ]:
print("=" * 60)
print("TESTING SET A: RAW FEATURES")
print("=" * 60)

# Define pipeline
pipeline_A = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ))
])

# Train
print(f"\n⏳ Training Random Forest on Set A...")
pipeline_A.fit(X_train_A, y_train_A)
print(f"✅ Training complete")

# Predict
y_pred_A = pipeline_A.predict(X_test_A)
y_pred_proba_A = pipeline_A.predict_proba(X_test_A)[:, 1]

# Calculate metrics
recall_A = recall_score(y_test_A, y_pred_A)
precision_A = precision_score(y_test_A, y_pred_A)
f1_A = f1_score(y_test_A, y_pred_A)
roc_auc_A = roc_auc_score(y_test_A, y_pred_proba_A)

print(f"\n📊 Set A Performance:")
print(f"   Recall:    {recall_A:.3f} ← PRIMARY METRIC")
print(f"   Precision: {precision_A:.3f}")
print(f"   F1-Score:  {f1_A:.3f}")
print(f"   ROC-AUC:   {roc_auc_A:.3f}")

# Confusion matrix
cm_A = confusion_matrix(y_test_A, y_pred_A)
tn_A, fp_A, fn_A, tp_A = cm_A.ravel()

print(f"\n📊 Confusion Matrix:")
print(f"   True Negatives:  {tn_A:6,}")
print(f"   False Positives: {fp_A:6,}")
print(f"   False Negatives: {fn_A:6,} ← Missed At-Risk")
print(f"   True Positives:  {tp_A:6,} ← Caught At-Risk")
print(f"\n   Catches {tp_A:,} out of {tp_A + fn_A:,} At-Risk patients ({recall_A*100:.1f}%)")

## 🧪 Step 7: Test Set B (Raw + Engineered Features)

In [ ]:
print("=" * 60)
print("TESTING SET B: RAW + ENGINEERED FEATURES")
print("=" * 60)

# Define pipeline
pipeline_B = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ))
])

# Train
print(f"\n⏳ Training Random Forest on Set B...")
pipeline_B.fit(X_train_B, y_train_B)
print(f"✅ Training complete")

# Predict
y_pred_B = pipeline_B.predict(X_test_B)
y_pred_proba_B = pipeline_B.predict_proba(X_test_B)[:, 1]

# Calculate metrics
recall_B = recall_score(y_test_B, y_pred_B)
precision_B = precision_score(y_test_B, y_pred_B)
f1_B = f1_score(y_test_B, y_pred_B)
roc_auc_B = roc_auc_score(y_test_B, y_pred_proba_B)

print(f"\n📊 Set B Performance:")
print(f"   Recall:    {recall_B:.3f} ← PRIMARY METRIC")
print(f"   Precision: {precision_B:.3f}")
print(f"   F1-Score:  {f1_B:.3f}")
print(f"   ROC-AUC:   {roc_auc_B:.3f}")

# Confusion matrix
cm_B = confusion_matrix(y_test_B, y_pred_B)
tn_B, fp_B, fn_B, tp_B = cm_B.ravel()

print(f"\n📊 Confusion Matrix:")
print(f"   True Negatives:  {tn_B:6,}")
print(f"   False Positives: {fp_B:6,}")
print(f"   False Negatives: {fn_B:6,} ← Missed At-Risk")
print(f"   True Positives:  {tp_B:6,} ← Caught At-Risk")
print(f"\n   Catches {tp_B:,} out of {tp_B + fn_B:,} At-Risk patients ({recall_B*100:.1f}%)")

## 📊 Step 8: Direct Comparison

In [ ]:
print("=" * 60)
print("A/B TEST RESULTS")
print("=" * 60)

# Create comparison dataframe
comparison = pd.DataFrame([
    {
        'Feature Set': 'Set A (Raw)',
        'N_Features': X_set_A.shape[1],
        'Recall': recall_A,
        'Precision': precision_A,
        'F1': f1_A,
        'ROC_AUC': roc_auc_A
    },
    {
        'Feature Set': 'Set B (Engineered)',
        'N_Features': X_set_B.shape[1],
        'Recall': recall_B,
        'Precision': precision_B,
        'F1': f1_B,
        'ROC_AUC': roc_auc_B
    }
])

print("\n📊 Performance Comparison:")
print(comparison.to_string(index=False))

# Calculate deltas
recall_improvement = recall_B - recall_A
recall_improvement_pct = (recall_improvement / recall_A) * 100
roc_improvement = roc_auc_B - roc_auc_A

print("\n" + "=" * 60)
print("IMPROVEMENT ANALYSIS")
print("=" * 60)

print(f"\n📈 Recall (Primary Metric):")
print(f"   Set A: {recall_A:.3f}")
print(f"   Set B: {recall_B:.3f}")
print(f"   Δ = {recall_improvement:+.3f} ({recall_improvement_pct:+.1f}%)")

if recall_improvement > 0:
    print(f"   ✅ Set B catches {int(recall_improvement * (tp_B + fn_B))} MORE At-Risk patients")
else:
    print(f"   ⚠️ Set B catches {int(abs(recall_improvement) * (tp_B + fn_B))} FEWER At-Risk patients")

print(f"\n📈 ROC-AUC (Secondary Metric):")
print(f"   Set A: {roc_auc_A:.3f}")
print(f"   Set B: {roc_auc_B:.3f}")
print(f"   Δ = {roc_improvement:+.3f}")

## 📊 Step 9: Visualization

In [ ]:
# Create comparison charts
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Recall comparison
sets = ['Set A\n(Raw)', 'Set B\n(Engineered)']
recalls = [recall_A, recall_B]
colors = ['steelblue', 'coral']

axes[0].bar(sets, recalls, color=colors, alpha=0.7)
axes[0].set_ylabel('Recall (At-Risk)', fontsize=11, fontweight='bold')
axes[0].set_title('Recall Comparison\n(Higher is Better)', fontsize=12, fontweight='bold')
axes[0].set_ylim(0, 1)
axes[0].axhline(y=0.70, color='green', linestyle='--', alpha=0.5, label='Target (70%)')
axes[0].legend()
for i, v in enumerate(recalls):
    axes[0].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold', fontsize=11)

# 2. Precision comparison
precisions = [precision_A, precision_B]
axes[1].bar(sets, precisions, color=colors, alpha=0.7)
axes[1].set_ylabel('Precision (At-Risk)', fontsize=11, fontweight='bold')
axes[1].set_title('Precision Comparison\n(Higher is Better)', fontsize=12, fontweight='bold')
axes[1].set_ylim(0, 1)
for i, v in enumerate(precisions):
    axes[1].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold', fontsize=11)

# 3. ROC-AUC comparison
roc_aucs = [roc_auc_A, roc_auc_B]
axes[2].bar(sets, roc_aucs, color=colors, alpha=0.7)
axes[2].set_ylabel('ROC-AUC', fontsize=11, fontweight='bold')
axes[2].set_title('ROC-AUC Comparison\n(Higher is Better)', fontsize=12, fontweight='bold')
axes[2].set_ylim(0, 1)
axes[2].axhline(y=0.5, color='red', linestyle='--', alpha=0.3, label='Random (0.5)')
axes[2].legend()
for i, v in enumerate(roc_aucs):
    axes[2].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.show()

## 🎯 Step 10: Decision Gate

In [ ]:
print("=" * 60)
print("DECISION GATE: FEATURE ENGINEERING A/B TEST")
print("=" * 60)

# Decision criteria
recall_threshold = 0.02  # 2% improvement required

print(f"\n📋 Decision Criteria:")
print(f"   If Set B improves Recall by >{recall_threshold:.1%}, use engineered features")
print(f"   Otherwise, use raw features (simpler is better if no gain)")

print(f"\n📊 Actual Results:")
print(f"   Recall improvement: {recall_improvement:+.3f} ({recall_improvement_pct:+.1f}%)")

# Make decision
if recall_improvement > recall_threshold:
    decision = "SET_B"
    selected_features = X_set_B
    selected_feature_names = X_set_B.columns.tolist()
    
    print(f"\n✅ DECISION: Use SET B (Engineered Features)")
    print(f"\n📋 Reasoning:")
    print(f"   • Recall improved by {recall_improvement:.3f} (>{recall_threshold:.2f} threshold)")
    print(f"   • Catches {int(recall_improvement * (tp_B + fn_B))} MORE At-Risk patients")
    print(f"   • ROC-AUC also improved by {roc_improvement:+.3f}")
    print(f"   • Feature engineering provides measurable value")
    
elif recall_improvement > 0:
    decision = "SET_A"
    selected_features = X_set_A
    selected_feature_names = X_set_A.columns.tolist()
    
    print(f"\n⚠️ DECISION: Use SET A (Raw Features)")
    print(f"\n📋 Reasoning:")
    print(f"   • Recall improved by only {recall_improvement:.3f} (<{recall_threshold:.2f} threshold)")
    print(f"   • Improvement too small to justify added complexity")
    print(f"   • Simpler model (fewer features) is preferable")
    print(f"   • Risk of overfitting with engineered features")
    
else:
    decision = "SET_A"
    selected_features = X_set_A
    selected_feature_names = X_set_A.columns.tolist()
    
    print(f"\n❌ DECISION: Use SET A (Raw Features)")
    print(f"\n📋 Reasoning:")
    print(f"   • Recall DECREASED by {abs(recall_improvement):.3f}")
    print(f"   • Engineered features HURT performance")
    print(f"   • Likely introduced noise or overfitting")
    print(f"   • Stick with raw features")

print(f"\n📊 Selected Feature Set:")
print(f"   Number of features: {len(selected_feature_names)}")
print(f"   Baseline recall: {recall_B if decision == 'SET_B' else recall_A:.3f}")

## 💾 Step 11: Save Decision for Next Phase

In [ ]:
print("=" * 60)
print("SAVING RESULTS FOR PHASE 2")
print("=" * 60)

# Save selected features
selected_features_df = selected_features.copy()
selected_features_df['Target'] = y
selected_features_df.to_csv('selected_features_05A.csv', index=False)

print(f"\n✅ Saved: selected_features_05A.csv")
print(f"   Shape: {selected_features_df.shape}")

# Save decision metadata
import json
decision_data = {
    'decision': decision,
    'feature_count': len(selected_feature_names),
    'feature_names': selected_feature_names,
    'set_A_recall': float(recall_A),
    'set_B_recall': float(recall_B),
    'recall_improvement': float(recall_improvement),
    'set_A_roc_auc': float(roc_auc_A),
    'set_B_roc_auc': float(roc_auc_B),
    'baseline_recall': float(recall_B if decision == 'SET_B' else recall_A)
}

with open('feature_engineering_decision.json', 'w') as f:
    json.dump(decision_data, f, indent=2)

print(f"\n✅ Saved: feature_engineering_decision.json")
print(f"   Decision: {decision}")
print(f"   Baseline Recall: {decision_data['baseline_recall']:.3f}")

print("\n" + "=" * 60)
print("🎯 PHASE 1 COMPLETE")
print("=" * 60)
print(f"\n✅ Ready for Phase 2: Model Tournament")
print(f"   Next notebook: 05B_model_tournament.ipynb")
print(f"   Will test 4 models on {decision} features")